In [23]:
import QuantLib as ql

valuationDate = ql.Date(24, 11, 2020)  
ql.Settings.instance().evaluationDate =valuationDate #평가일자가 이 날이다~

calendar = ql.SouthKorea()
dayCount = ql.ActualActual()

#Simple Quote Object
underlying_qt = ql.SimpleQuote(270.48)  #기초자산 가격 #kospi200 주가 (예시)
dividend_qt = ql.SimpleQuote(0.0) #배당없다고 가정
interestRate_qt = ql.SimpleQuote(0.01)  #1% (실제로는 이자율스왑커브 가져와서 넣음)
volatility_qt = ql.SimpleQuote(0.13) #13%

#Quote Handles
u_qhd = ql.QuoteHandle(underlying_qt)
d_qhd = ql.QuoteHandle(dividend_qt)
r_qhd = ql.QuoteHandle(interestRate_qt)
v_qhd = ql.QuoteHandle(volatility_qt)

#배당, 금리, 변동성은 만기에 따른 기간구조가 존재
#구현의 편의성을 위하여 전 만기에서 모두 동일한 값이라고 가정함
# Term Structure 
r_ts = ql.FlatForward(valuationDate, r_qhd, dayCount) # 이값이 전 만기에 대해 동일한 값을 가진다.
d_ts = ql.FlatForward(valuationDate, d_qhd, dayCount)
v_ts = ql.BlackConstantVol(valuationDate, calendar, v_qhd, dayCount)

# TermStructure Handle
r_tshd = ql.YieldTermStructureHandle(r_ts)
d_tshd = ql.YieldTermStructureHandle(d_ts)
v_tshd = ql.BlackVolTermStructureHandle(v_ts)

# Process
#유로피안 옵션에 대해 주로 사용하는 Process
process = ql.BlackScholesMertonProcess(u_qhd, d_tshd, r_tshd, v_tshd)

# Engine
engine = ql.AnalyticEuropeanEngine(process)

# Option Obj
option_type = ql.Option.Call
strikePrice = 272 # 주가 272 넘으면 이익
expiryDate = ql.Date(24, 11, 2021)
exercise = ql.EuropeanExercise(expiryDate)
payoff = ql.PlainVanillaPayoff(option_type, strikePrice)  #일반적인 payoff

option = ql.VanillaOption(payoff, exercise)
option.setPricingEngine(engine)

#프라이싱된 옵션의 이론적 가치와 그릭스
print("Option Premium : {}".format(option.NPV()))
print("Delta : {}".format(option.delta()))
print("theta : {}".format(option.thetaPerDay()))
print("gamma : {}".format(option.gamma()))
print("vega : {}".format(option.vega() / 100))
print("rho : {}".format(option.rho() / 100))

underlying_qt.setValue(275) #기초자산 가격 변동
print()
print("기초자산 가격 변동 setValue해주면 알아서 값 다르게 해줌")
print("***New : Underlying Asset Value : 270.48 -> 275\n")
print("Option Premium : {}".format(option.NPV()))
print("Delta : {}".format(option.delta()))
print("theta : {}".format(option.thetaPerDay()))
print("gamma : {}".format(option.gamma()))
print("vega : {}".format(option.vega() / 100))
print("rho : {}".format(option.rho() / 100))


print()

#내재변동성 Implied Volatility
underlying_qt.setValue(270.48)
mkt_price = 8.21 # 이 옵션의 시가
#임의로 가정한 13%가 아닌 이 옵션의 시가에 가정된.. 시장참여자들이 생각하는 변동성
implied_vol = option.impliedVolatility(mkt_price, process)
print("시장은 만기까지의 내재변동성 수준을", implied_vol*100, '%으로 봄')

#내재변동성으로 재수정
print()
volatility_qt.setValue(implied_vol)
print("Option Premium : {}".format(option.NPV()))
if round(option.NPV(),2) == mkt_price:
    print("\t\t\t위에서 정한 mkt_price와 같은 수준\n")
print("Delta : {}".format(option.delta()))
print("theta : {}".format(option.thetaPerDay()))
print("gamma : {}".format(option.gamma()))
print("vega : {}".format(option.vega() / 100))
print("rho : {}".format(option.rho() / 100))

Option Premium : 14.586000963056717
Delta : 0.5393474061232754
theta : -0.022722469700332086
gamma : 0.01129208403778226
vega : 1.0736536768091984
rho : 1.312593377840626

기초자산 가격 변동 setValue해주면 알아서 값 다르게 해줌
***New : Underlying Asset Value : 270.48 -> 275

Option Premium : 17.13792979095591
Delta : 0.5895128624633252
theta : -0.023018071007202906
gamma : 0.01087867744146228
vega : 1.0692057512531787
rho : 1.4493686801164989

시장은 만기까지의 내재변동성 수준을 7.063398966105272 %으로 봄

Option Premium : 8.209995514675025
			위에서 정한 mkt_price와 같은 수준

Delta : 0.5388425504562017
theta : -0.014160920111173697
gamma : 0.02078537774962862
vega : 1.0737876902800079
rho : 1.3749701504431195
